PROJECT 3

Part 1: Prediction quality vs feature selection

In [150]:
import numpy as np
from sklearn.linear_model import LassoCV, Lasso
from pandas import DataFrame
from sklearn.model_selection import train_test_split
import random
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

Hyperparameter $\lambda$ are instead $\alpha$ in LassoCV

`alpha_` contains hyperparameter that achieved lowest MSE

`alphas_` contains all tested hyperparameters

Coeffs of `Lasso` can be found in `coef_`

`mse_path_` a `(len(alphas_) x n_folds)` matrix storing test error on each fold

Vectors **e** and **s** can easily be computed from `mse_path_`

In [8]:
# EXAMPLE CODE FROM THESIS

# cv_mean = np.mean(Lasso.mse_path_, axis=1)
# cv_std = np.std(Lasso.mse_path_, axis=1)
# idx_min_mean = np.argmin(cv_mean)
# idx_alpha = np.where(
#     (cv_mean <= cv_mean[idx_min_mean] + cv_std[idx_min_mean] / np.sqrt(n_folds)) &
#     (cv_mean >= cv_mean[idx_min_mean])
# )[0][0]
# alpha_1se = Lasso.alphas_[idx_alpha]

In [2]:
# Function to simulate data from the thesis
def simulate_data(n, p, rng, sparsity=0.95, SNR=2.0, beta_scale=5.0):
    """Simulate data for Project 3, Part 1.

    Parameters
    ----------
    n : int
        Number of samples
    p : int
        Number of features
    rng : numpy.random.Generator
        Random number generator (e.g. from `numpy.random.default_rng`)
    sparsity : float in (0, 1)
        Percentage of zero elements in simulated regression coefficients
    SNR : positive float
        Signal-to-noise ratio (see explanation above)
    beta_scale : float
        Scaling for the coefficient to make sure they are large

    Returns
    -------
    X : `n x p` numpy.array
        Matrix of features
    y : `n` numpy.array
        Vector of responses
    beta : `p` numpy.array
        Vector of regression coefficients
    """
    X = rng.standard_normal(size=(n, p))
    
    q = int(np.ceil((1.0 - sparsity) * p))
    beta = np.zeros((p,), dtype=float)
    beta[:q] = beta_scale * rng.standard_normal(size=(q,))
    
    sigma = np.sqrt(np.sum(np.square(X @ beta)) / (n - 1)) / SNR

    y = X @ beta + sigma * rng.standard_normal(size=(n,))

    # Shuffle columns so that non-zero features appear
    # not simply in the first (1 - sparsity) * p columns
    idx_col = rng.permutation(p)
    
    return X[:, idx_col], y, beta[idx_col]

Suggestions for parameters:<br>
- `p` - something large, e.g. `500` or `1000`<br>
- `n` - vary compared to p, e.g. iterate through `[200, 500, 750]` if `p = 1000`<br>
what truly matters is the ratio `p/n`, <br>
if you change `p`, adjust choices of `n`<br>
- `sparsity` - vary for a few choices e.g. `[0.75, 0.9, 0.95, 0.99]`<br>
- `SNR` - fix at something reasonable like `2` or `5`<br>
- `beta_scale` - fix at maybe `5` or `10`<br>

Questions
- How des the MSE of the $\lambda_{min}$ and $\lambda_{1se}$ models behave for different $n$ and sparsity levels?
- What differences between sensitivity/specificity computed from the $\lambda_{min}$ and the $\lambda_{1se}$ models can you observe?
- How do different choices for `n` and `sparisty` affect the relationship of sensitivity/specifity?

In [90]:
p = 500 # 500
nmin = 100 # 100
nmax = p-nmin
nn = 3# 3
n = np.linspace(nmin, nmax, nn)
sparsity = [0.75, 0.9, 0.95, 0.99]
# sparsity = [0.75]
SNR = 2
beta_scale = 5
print(n)

[100. 250. 400.]


In [188]:
rng = np.random.default_rng()
n_runs = 3
n_folds = 5
train_frac = 0.8
# Dictionary with the alpha_min and alpha_1se.
# Each list will contain the data for each value of n
MSEtrain = {'min':[], '1se':[]}
MSEtest = {'min':[], '1se':[]}

# With len(n)=3 and len(sparsity)=4 and 5 repeats,
# we will run thourgh 60 datasets, be aware
for cn in n:
    MSE_train_list = [[], []]
    MSE_test_list = [[], []]
    for csparsity in sparsity:
        MSE_train_1se = []
        MSE_train_min = []
        MSE_test_1se = []
        MSE_test_min = []
        for _ in range(n_runs):
            X, y, beta = simulate_data(int(cn), p, rng, csparsity, SNR, beta_scale)
            train_idx = int(np.floor(cn*train_frac))
            # split into training and test
            trainX = X[0:train_idx]
            trainy = y[0:train_idx]
            testX = X[train_idx:]
            testy= y[train_idx:]
            # fit lassoCV to training data
            lasso = LassoCV(cv=n_folds).fit(trainX, trainy) #, random_state=0
            beta_est = lasso.coef_
            non_zero_idx = np.nonzero(beta_est)[0]
            selected_features_est = deepcopy(beta_est)   
            selected_features_est[non_zero_idx] = 1  
            cv_mean = np.mean(lasso.mse_path_, axis=1)
            cv_std = np.std(lasso.mse_path_, axis=1)
            idx_min_mean = np.argmin(cv_mean)
            idx_alpha = np.where(
                (cv_mean <= cv_mean[idx_min_mean] + cv_std[idx_min_mean] / np.sqrt(n_folds)) &
                (cv_mean >= cv_mean[idx_min_mean])
            )[0][0]
            alpha_1se = lasso.alphas_[idx_alpha]
            # extract data of MSE from alpha_min and alpha_1se
            MSE_train_min.append(cv_mean[idx_min_mean])
            MSE_train_1se.append(cv_mean[idx_alpha])

            # predict test data
            test_pred_min = lasso.predict(testX)
            MSE_test_min.append(mean_squared_error(testy, test_pred_min))           

            # refit on training data with specific alpha and predict on test data
            upd_lasso = Lasso(alpha_1se).fit(trainX, trainy)
            test_pred_1se = upd_lasso.predict(testX)
            MSE_test_1se.append(mean_squared_error(testy, test_pred_1se))
        
            



        MSE_train_list[0].append(MSE_train_min)
        MSE_train_list[1].append(MSE_train_1se)
        MSE_test_list[0].append(MSE_test_min)
        MSE_test_list[1].append(MSE_test_1se)
    MSEtrain['min'].append(MSE_train_list[0])
    MSEtrain['1se'].append(MSE_train_list[1])
    MSEtest['min'].append(MSE_test_list[0])
    MSEtest['1se'].append(MSE_test_list[1])



[-0.          0.          0.         -0.         -0.         -0.
 -0.          0.         -0.          0.         -0.          0.
 -0.          0.         -0.          0.          0.          0.
  0.         -0.          0.          0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.          0.          0.          0.
  0.         -0.         -0.         -0.         -0.          0.
 -0.         -0.         -0.          0.         -0.          0.
  0.          0.          0.         -4.89154691 -0.          0.
  0.          0.         -0.         -0.          0.         -0.
  0.         -0.         -0.          0.          0.          0.
  0.          0.          0.         -0.         -0.          0.
 -0.         -0.         -0.         -0.          0.         -0.
 -0.          0.         -0.         -0.          0.         -0.
  0.         -0.          0.          0.         -0.          0.
  0.          0.         

KeyboardInterrupt: 

In [209]:
a = np.array([0.4, 0.1, 0, 3])
b = np.nonzero(a)[0]
print(a)
print(b)
a[b] = 1
print(a)



[0.4 0.1 0.  3. ]
[0 1 3]
[1. 1. 0. 1.]


In [185]:
# train data
# c = ['red', 'blue', 'green']
# for j in range(4):
#     plt.figure(j)
#     for i in range(len(n)):
#         min_means = np.mean(MSEtrain['min'][i], axis=1)
#         min_std = np.std(MSEtrain['min'][i], axis=1)
#         min_stds = [min_means - min_std,
#                     min_means + min_std]
#         line, = plt.plot(sparsity, min_means, c[i])
#         line.set_label(int((n[i])))
#         plt.plot(sparsity, min_stds[0], c[i], linestyle='dashed')
#         plt.plot(sparsity, min_stds[1], c[i], linestyle='dashed')
#     plt.title('MSE_min train')
#     plt.xlabel('sparsity')
#     plt.ylabel('MSE')
#     plt.legend()

In [187]:
print(lasso.coef_)

0.0
